In [ ]:
import requests  # Using this library to access HTTP/HTTPs through python, receive our images/videos from the web,
# pretty much anything that has to do with python and the internet relies on this library!
import re  # Using this mainly to verify user input URLs or correct them to get the perfect input to match the requests function
from datetime import datetime  # The program names files by generating timestamps of downloading the file! Hence this
# is needed to obtain date and time for the file naming purposes
from tqdm import tqdm  # Just added as a base/outline to help make the loading bar in the actual GUI, as both of them
# use a similar library anyways (tqdm for code progress bars and tqdm.gui for gui progress bars)
# Progress bars help because they can 1- Let the user know if theyre really downloading something or stuck
# 2- Let them know how long they'll wait for, especially if its a video file that can be large
import instaloader
from instaloader import Post  # This establishes a connection with Instagram API, because if you access
#instagram as a bot, they have bot detection measures and block you out, hence we use the API provided!
L = instaloader.Instaloader()

# Code Intro
print(''' 

            XMUM AIT102 22/04 Python Group 3 Present:
                        InstaGrab V1.0
    Omar Mahmoud ~ Yap Wei Herng ~ Wong Zhan Song ~ Chia Xiu Xiang

''')
USER = input("Enter your username: ")
PASSWORD = input("Enter your password: ")
L.login(USER, PASSWORD)        # (login)
print("Logged in successfully!")

# This function is just to test if internet works for user, if doesnt work we close the program.
# (Running the program with no internet connection can lead to disaster)
def connection(url='http://www.youtube.com',
  timeout=5):  # Using youtube.com as the test website, as almost never goes down
    try:
        req = requests.get(url, timeout=timeout)
        req.raise_for_status()
        print("Internet Connection successful!\n")
        return True
    except requests.HTTPError as e:
        print("Checking internet connection failed, status code {0}.".format(
            e.response.status_code))
    except requests.ConnectionError:
        print("You aren't connected to the internet!")
    return False


# The InstaGrab primary function, to download both video and images!
def download_video():
    url = input("Please enter instagram URL: ")
    x = re.match(r'^(https:)[/][/]www.([^/]+[.])*instagram.com',
                 url)  # makes sure URL is truly an instagram link, in the media format as image or video.
    #  If instagram ever updates their media link format we need to change this accordingly
    shortcode = re.search('(?:https?:\/\/)?(?:www.)?instagram.com\/?([a-zA-Z0-9\.\_\-]+)?\/([p]+)?([reel]+)?([tv]+)?([stories]+)?\/([a-zA-Z0-9\-\_\.]+)\/?([0-9]+)?',url)
    shortcode = shortcode.group(6)
    post = Post.from_shortcode(L.context, shortcode)


    try:
        if x:  # x is the verified/correctly formatted URL
            request_image = requests.get(url)
            src = request_image.content.decode('utf-8')  # page source obtaining function2
            msg = input("You are trying to download a video. Do you want to continue? (Yes or No): ".lower())
            if msg == "yes":
                print("Downloading the video...")
                L.download_post(post, target="Downloads")
                extract_video_link = re.search(r'<video class="_ab1d"*src="()"', src) # Scans for video link
                video_link = extract_video_link.group(1) # Captures whats in the parenthesis
                final = re.sub('<video class="_ab1d"*src="', '', video_link) # Puts video link into proper format
                _response = requests.get(final).content
                file_size_request = requests.get(final, stream=True)
                file_size = int(file_size_request.headers['Content-Length'])
                block_size = 1024
                filename = datetime.strftime(datetime.now(), '%Y-%m-%d-%H-%M-%S')
                t = tqdm(total=file_size, unit='B', unit_scale=True, desc=filename, ascii=True)
                with open(filename + '.mp4', 'wb') as f:
                    for data in file_size_request.iter_content(block_size):
                        t.update(len(data))
                        f.write(data)
                t.close()

                print("Video downloaded successfully")

            if msg == "no":
                exit()
        else:
            print("Entered URL is not an instagram.com URL.")
    except:
            print("Done!")

def download_image():
    url = input("Please enter instagram URL: ")
    x = re.match(r'^(https:)[/][/]www.([^/]+[.])*instagram.com',
                 url)  # makes sure URL is truly an instagram link, in the
    # media format as image or video. If instagram ever updates their media link format we need to change this accordingly
    shortcode = re.search(
        '(?:https?:\/\/)?(?:www.)?instagram.com\/?([a-zA-Z0-9\.\_\-]+)?\/([p]+)?([reel]+)?([tv]+)?([stories]+)?\/([a-zA-Z0-9\-\_\.]+)\/?([0-9]+)?',url)
    shortcode = shortcode.group(6)
    post = Post.from_shortcode(L.context, shortcode)

    try:
        if x:  # x is the verified/correctly formatted URL
            request_image = requests.get(url)
            src = request_image.content.decode('utf-8')  # page source obtaining function2
            msg = input("You are trying to download an image. Do you want to continue? (Yes or No): ".lower())
            if msg == "yes":
                print("\nDownloading the image...")
                L.download_post(post, target="Downloads")
                extract_image_link = re.search(r'<img alt=*src="()"', src)
                image_link = extract_image_link.group()
                final = re.sub('<img alt=*src="()', '', image_link)
                _response = requests.get(final).content
                file_size_request = requests.get(final, stream=True)
                file_size = int(file_size_request.headers['Content-Length'])
                block_size = 1024
                filename = datetime.strftime(datetime.now(), '%Y-%m-%d-%H:%M:%S')
                t = tqdm(total=file_size, unit='B', unit_scale=True, desc=filename, ascii=True)
                with open(filename + '.jpg', 'wb') as f:
                    for data in file_size_request.iter_content(block_size):
                        t.update(len(data))
                        f.write(data)
                t.close()
                print("Image downloaded successfully")

            if msg == "no":
                exit()
        else:
            print("Entered URL is not an instagram.com URL.")
    except:
        print("Done!")



if connection() == True:  # This is the main function of program, uses both connection test and instagrab function to make it work.
    try:
        while True:
            a = "Press '1' to download an instagram photo.\nPress '2' to download an instagram video.\nPress 'Q' to exit program."
            print(a)
            select = str(input("\nInstaGrab > ")).upper()
            try:
                if select == '1':
                    download_image()
                if select == '2':
                    download_video()
                if select == 'Q':
                    quit()
                else:
                    quit()
            except (KeyboardInterrupt):
                print("Program Interrupted")
    except(KeyboardInterrupt):
        print("\nProgramme Interrupted")
else:
    quit()
